In [2]:
import numpy as np 
import pandas as pd
import json 
import requests 
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import lxml


source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
canada_list = BeautifulSoup(source, 'lxml') 
toronto_list = pd.DataFrame(columns=['Postalcode','Borough', 'Neighborhood'])

content = canada_list.find('div', class_='mw-parser-output')
table = content.table.tbody 
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0: 
            postcode = td.text
            i = i + 1
        elif i == 1: 
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','') 
    toronto_list = toronto_list.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
    
toronto_list = toronto_list[toronto_list.Borough!='Not assigned']
toronto_list = toronto_list[toronto_list.Borough!= 0]
toronto_list.reset_index(drop = True, inplace = True)

#
i = 0
for i in range(0,toronto_list.shape[0]):
    if toronto_list.iloc[i][2] == 'Not assigned':
        toronto_list.iloc[i][2] = toronto_list.iloc[i][1]
        i = i+1
        
#print the dataframe
df = toronto_list.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head(11)

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn
5,M1H\n,Scarborough\n,Cedarbrae
6,M1J\n,Scarborough\n,Scarborough Village
7,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L\n,Scarborough\n,"Golden Mile, Clairlea, Oakridge"
9,M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West"
